FIT5196 Assessment 1 

Student Name: VADDE PAVAN KUMAR 

Student ID: 29772990 

Date: 14/04/2019

Version: 3.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

In [596]:
# Loading the packages
import re 
import PyPDF2 
import nltk
from nltk.tokenize import MWETokenizer
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.collocations import *
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# ********Extracting the data from the PDF *******

units = [] # list to append all the data coming from PDF and storing in the list 
unit_code = [] #extracting the unit codes, from the PDF and storing them in the list 
unit_info = [] # extracting synopsis and outcomes and storing them in the list 

pdfFileObj = open('29772990.pdf', 'rb')   # Opening the PDF file to read the data and storing it in a variable 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)# using the package 
length = pdfReader.numPages
for i in range(length):
    pageObj = pdfReader.getPage(i)  # reading the page by page 
    word_data = pageObj.extractText() # reading the Text from the pages
    units.append(word_data)# appendinbg the extracted text to the units list 
# here we are performing these steps to extract the unitcodes and unit information 
l1 = [] 
for unit in units:
    l1.append(unit.split("\n"))  #splitting the strings with the respect to new line and storing in the list 
text = ""
for u in l1:
    for l2 in u: # we are jsut ignoring these headings in to the list 
        if l2 == "Title" or l2 == "Synopsis" or l2=="Outcomes":
            continue
        elif re.match(r'^([A-Z]{3}[0-9]{4})',l2): # we are matching the unitcodes 
            if len(l2) == 7: # there are many other units in the text so getting only the unit codes 
                unit_code.append(l2)
                if len(text) > 0: # appeding units infformation to the list unit_info 
                    unit_info.append(text) #appending the text to the list 
                    text = ""
            else:
                if len(l2) > 0: # we are performing the normalization and converting the first word of each line to lower case 
                    l = l2.split(" ")
                    l[0] = l[0].lower() # picking the only first element and converting it to lower case 
                    l2 = ' '.join(str(elem) for elem in l) #we are rejoining the strings back to make space at the end of each line 
                    if len(text) == 0:
                        text = text + l2
                    else:
                        text = text + " "+ l2
        else: # same process is repeated, if there is no match of unitcode and then will be stored into unit info 
            if len(l2) > 0:
                l = l2.split(" ")
                l[0] = l[0].lower()
                l2 = ' '.join(str(elem) for elem in l)
                if len(text) == 0:
                    text = text + l2
                else:
                    text = text + " "+ l2             
                
unit_info.append(text) #This we are appending outside the loop because, its the last units info

# ************Tokenizing and removing the stop words (Context independent)***************

unigram_tokens_list = []
stopped_tokens = []
stem_tokens = []
for info in unit_info: 
    tokenizer = RegexpTokenizer(r"\w+(?:[-.]\w+)?") # Tokenizing the units info with the given regex 
    unigram_tokens = tokenizer.tokenize(info) 
    unigram_tokens_list.append(unigram_tokens) # tokenized words are stored into the list unit wise 
    
    stopped_token = []
    with open("stopwords_en.txt") as fp: # removing the stop words as for the given stop words file 
        stop_words = fp.read().splitlines()
        for w in unigram_tokens: 
            if w not in stop_words: 
                stopped_token.append(w) # if word not in the stop words then we are appending to the stooped tokens list
        stopped_tokens.append(stopped_token)

# *************Removing context dependent stop wordsand the tokens with the length  less than 3 **********

high = []
low = []
cleaned_tokens = []
list1=[item for sublist in stopped_tokens for item in sublist] # this counter caluclate sthe frquencies of each word 
a = dict(Counter(list1)) # Converting the counter lsit to the dict object 
for key in a.keys():  # fectching all the keys 
    if a[key] > 190: # we are checking whether the frequency of any data is greater than 190
        high.append(key) # we are appending the word to the list named high 
    elif a[key] < 10:  # we are checking whether the frequency of any data is less than 10
        low.append(key)  # we are appending the word to the list named low for low frequencied words 
# we are having 2 high frequency words ('unit', 'research')
# we are having of 3641 words with less frequency 
merge = high + low # now we are merging both the lists to single lists so we can remove the high frequency and low frequency words 
for token in stopped_tokens: # now we are removing the things from the stopped tokes 
    clean = [] 
    for w in token: # iterating word by word
        if w not in merge: # if the word of the unit is in the list then it will be excluded 
            if len(w) > 3: # and also checking the condition whether the token length is greater than 3 
                 clean.append(w)
    cleaned_tokens.append(clean)

# *******************Applying porter stemmer to the tokens ******************

stemmer = PorterStemmer() # intializing the PorterStemmer to the stemmer as short form 
final_conv_tokens =[] # list to store alll the final stemmed tokens 
### write your code below
for s in cleaned_tokens: # getting the list of words from cleaned tokens 
    conv_tokens = []# stemmed tokens are being stored in list unit wise
    for w in s:
        cw = stemmer.stem(w) # stemming each word in the unit using porter stemmer
        conv_tokens.append(cw)
    final_conv_tokens.append(conv_tokens) # storing all the stemmed tokes unit wise into the final_conv_tokens 

flattened_list = [y for x in final_conv_tokens for y in x] # merging all the lists into single list for finding bigram measures 
bigram_measures = nltk.collocations.BigramAssocMeasures() # importing bigram measure and bigram finder 
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(flattened_list) # importing the igram finder 
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-200 bigrams

# we need to make sure those collocations are not split into two individual words
mwetokenizer = MWETokenizer(top_200_bigrams)
colloc_info = [mwetokenizer.tokenize(info) for info in cleaned_tokens]

# *******************Writing the Unigrams and bigrams to the file ********************

combine_info = [j for i in colloc_info for j in i] # combining the lists into single list 
combine_info = set(combine_info)  # removing the duplicates from the list 
combine_info = sorted(combine_info) # sorted the words alphabetically 
save_file = open("29772990_vocab.txt", 'w') # opening the file to write the data into file 
save_file.write("Unigram_Tokens") # writing all the bigram tokens and appending the index number 
i = 0
for t in combine_info:
    save_file.write(t)
    save_file.write(":")
    save_file.write(str(i))
    save_file.write("\n")
    i = i+1

save_file.write("\n")
save_file.write("Bigram_Tokens")
for bigram in top_200_bigrams: # writing the top 200 bigrams to the file 
    save_file.write(str(bigram))
    save_file.write("\n")
save_file.close()

# ************************** Writing the sparse representations to the file *************************

save_file = open("29772990_countVec.txt", 'w') # opening the file to write the vectors array 
vectorizer = TfidfVectorizer()
for i in range(len(unit_code)):
    # list of text documents
    text = colloc_info[i]
    # create the transform 
    vectorizer = CountVectorizer() # using the count vector to count the no frequency of each word in the unit info 
    # tokenize and build vocab
    vectorizer.fit_transform(text) 
    # summarize
    a = vectorizer.vocabulary_
    save_file.write(unit_code[i]) # writing to the file as specified format
    save_file.write(",")
    for b in a.keys():
        save_file.write(b)
        save_file.write(":")
        save_file.write(str(a[b]))
        save_file.write(",")
    save_file.write("\n")
save_file.close()